In [29]:
import utils
import importlib
import polars as pl

importlib.reload(utils)

DATA_PATH = "./data/"

LIMIT_DATA = 100000

social_graph = utils.SocialGraph(source_file=DATA_PATH + "social_graph.csv")
social_graph.build(limit=LIMIT_DATA)



dataset = utils.CandidateDataset(
    source_train=DATA_PATH + "train.csv",
    source_test=DATA_PATH + "test.csv",
    source_feature_data=DATA_PATH + "feature_metadata.json",
)


dataset.build_datasets(limit=None)
dataset.df_train.sort(by="ID").head()

Successfully built social graph with 104023 nodes.
Imported train dataset with 272819 entries (272819 of which have missing features)
Imported test dataset with 48416 entries (15162 of which have missing features)


ID,feature_001,feature_002,feature_003,feature_004,feature_005,feature_006,feature_007,feature_008,feature_009,feature_010,feature_011,feature_012,feature_013,feature_014,feature_015,feature_016,feature_017,feature_018,high_conf_clean,is_cheating
u64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64
111395511279975,2.0,2.0,5.0,8.0,1.0,1.0,1.0,10.0,10.0,687.0,0.0,0.0,1.0,1.0,0.23463,5,22.203337,0.48195,1.0,null
149515486601323,2.0,3.0,7.0,1.0,2.0,3.0,1.0,9.0,9.0,65.0,0.0,0.0,0.0,1.0,0.028935,0,null,0.82,1.0,null
160906985161287,5.0,7.0,8.0,1.0,8.0,7.0,1.0,9.0,9.0,762.0,0.0,0.0,1.0,1.0,1.065972,2,5.725,0.3902,null,0.0
372923524293330,null,null,null,4.0,6.0,7.0,1.0,10.0,9.0,810.0,0.0,0.0,1.0,1.0,0.036042,1,1.233333,0.65,1.0,null
396236678209396,4.0,5.0,8.0,8.0,5.0,5.0,1.0,9.0,0.0,235.0,0.0,0.0,1.0,1.0,0.931875,2,15.325,0.75,1.0,null


In [30]:
dataset.apply_feature_typing()
dataset.df_train.sort(by="ID").head()

Applying feature types to datasets...


ID,feature_001,feature_002,feature_003,feature_004,feature_005,feature_006,feature_007,feature_008,feature_009,feature_010,feature_011,feature_012,feature_013,feature_014,feature_015,feature_016,feature_017,feature_018,high_conf_clean,is_cheating
u64,cat,i8,i8,i8,i8,i8,bool,i8,i8,f32,bool,cat,bool,bool,f32,f32,f32,f32,bool,bool
111395511279975,"""2""",2,5,8,1,1,true,10,10,687.0,false,"""0""",true,true,0.23463,5.0,22.203337,0.48195,true,false
149515486601323,"""2""",3,7,1,2,3,true,9,9,65.0,false,"""0""",false,true,0.028935,0.0,13.805947,0.82,true,false
160906985161287,"""5""",7,8,1,8,7,true,9,9,762.0,false,"""0""",true,true,1.065972,2.0,5.725,0.3902,false,false
372923524293330,"""missing""",1,1,4,6,7,true,10,9,810.0,false,"""0""",true,true,0.036042,1.0,1.233333,0.65,true,false
396236678209396,"""4""",5,8,8,5,5,true,9,0,235.0,false,"""0""",true,true,0.931875,2.0,15.325,0.75,true,false


In [31]:
dataset.dummy_encode_categorical()
dataset.df_train.sort(by="ID").head()

ID,feature_002,feature_003,feature_004,feature_005,feature_006,feature_007,feature_008,feature_009,feature_010,feature_011,feature_013,feature_014,feature_015,feature_016,feature_017,feature_018,high_conf_clean,is_cheating,feature_001:1,feature_001:2,feature_001:3,feature_001:4,feature_001:5,feature_012:0,feature_012:1,feature_012:2,feature_012:3
u64,i8,i8,i8,i8,i8,bool,i8,i8,f32,bool,bool,bool,f32,f32,f32,f32,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
111395511279975,2,5,8,1,1,true,10,10,687.0,false,true,true,0.23463,5.0,22.203337,0.48195,true,false,false,true,false,false,false,true,false,false,false
149515486601323,3,7,1,2,3,true,9,9,65.0,false,false,true,0.028935,0.0,13.805947,0.82,true,false,false,true,false,false,false,true,false,false,false
160906985161287,7,8,1,8,7,true,9,9,762.0,false,true,true,1.065972,2.0,5.725,0.3902,false,false,false,false,false,false,true,true,false,false,false
372923524293330,1,1,4,6,7,true,10,9,810.0,false,true,true,0.036042,1.0,1.233333,0.65,true,false,false,false,false,false,false,true,false,false,false
396236678209396,5,8,8,5,5,true,9,0,235.0,false,true,true,0.931875,2.0,15.325,0.75,true,false,false,false,false,true,false,true,false,false,false


In [32]:
import pandas as pd
import lace

# Create an engine from a dataframe
engine = lace.Engine.from_df(dataset.df_train)

# Fit a model to the dataframe over 5000 steps of the fitting procedure
engine.update(100)

# Show the statistical structure of the data -- which features are likely
# dependent (predictive) on each other
engine.clustermap("depprob", zmin=0, zmax=1)


0it [00:00, ?it/s]

In [35]:
engine.clustermap(
    'depprob',
    color_continuous_scale='greys_r',
    zmin=0,
    zmax=1
).figure.show()

In [39]:
engine.surprisal("is_cheating")\
    .sort("surprisal", descending=True)\
    .head(10)

index,is_cheating,surprisal
str,bool,f64
"""1030121925134133926""",true,6.174659
"""992210642512411605""",true,4.859761
"""107972236681305432""",true,4.839421
"""17733756113399007547""",true,4.162593
"""2174901037950354259""",true,4.047205
"""262852132057348071""",false,3.480158
"""1522603815227987432""",true,3.322786
"""9716188796485693826""",true,3.2953
"""18351371138756860861""",false,3.267574
